In [1]:
import calendar
import csv
from datetime import datetime, timedelta
import itertools
import os
import pprint

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import matplotlib.ticker as ticker
import numpy as np
import pandas as pd

plt.rcParams.update({'font.size': 8})


In [250]:
et_cell_path = 'et_cell_metadata.csv'
# et_cell_path = os.path.join(os.getcwd(), 'et_cell_metadata.csv')

kc_ws = os.path.join(os.getcwd(), 'kc')
eto_ws = os.path.join(os.getcwd(), 'eto')
etf_ws = os.path.join(os.getcwd(), 'etf')

eto_file_prefix = 'eto'
etf_file_prefix = 'etf'

interp_days = 32

# Don't include 1999 or 2013 because they only have partial 2 Landsat coverage
# Don't include 2022 or 2023 for now since the Kc data ends Dec 2022
# Not sure if 2012 is really needed in the one Landsat list, but including it to get the SLC-off scenes
# For the two Landsat list, started with the even years then added a few to match the one Landsat count
one_landsat_years = [1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 2012]
two_landsat_years = [2000, 2001, 2002, 2004, 2005, 2006, 2008, 2009, 2010, 2014, 2015, 2016, 2018, 2019, 2020]
landsat_years = one_landsat_years + two_landsat_years
print(one_landsat_years)
print(two_landsat_years)

summer_months = [6, 7, 8]
growsn_months = [4, 5, 6, 7, 8, 9, 10]

crop_types = {
    3: 'Alfalfa Hay',
    4: 'Grass Hay',
    7: 'Field Corn',
    13: 'Winter Grain',
    16: 'Grass Pasture',
    19: 'Apples and Cherries',
    # 25: 'Grapes, Wine',
    42: 'Blue Grass Seed',
}
crops = [3, 4, 7, 13, 16, 19, 25, 42]


[1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 2012]
[2000, 2001, 2002, 2004, 2005, 2006, 2008, 2009, 2010, 2014, 2015, 2016, 2018, 2019, 2020]


### Read ET cell metadata file

In [251]:
print('')
et_cell_df = pd.read_csv(et_cell_path)
print(f'  ET cell count: {len(et_cell_df)}')
print(et_cell_df)


  ET cell count: 11
    GRIDMET_ID        LAT         LON     ELEV_M   ELEV_FT  FIPS_C STPO  \
0       387098  36.691661 -107.933323  1701.0400  5580.840   35045   NM   
1       467476  39.108327 -108.349989  1513.2000  4964.570    8077   CO   
2       473008  39.274994 -108.849989  1426.9200  4681.500    8077   CO   
3       511799  40.441660 -109.558322  1679.2400  5509.320   49047   UT   
4       581084  42.524994 -110.183322  2130.3600  6989.370   56035   WY   
5       605776  43.275000 -120.850000  1314.7200  4313.390   41037   OR   
6       661167  44.941667 -122.891667    69.1667   226.925   41047   OR   
7       686149  45.691667 -121.475000   231.8400   760.630   41027   OR   
8       688961  45.775000 -119.808333   179.2000   587.927   41049   OR   
9       564161  42.024995 -122.308319  1116.6000  3663.390   41029   OR   
10      565566  42.066661 -121.516653  1370.3600  4495.930   41035   OR   

    COUNTYNAME CNTYCATEGO   STATENAME      HUC8  
0     San Juan     County  N

## Show ET cell locations

In [252]:
# import ee
# import geemap
# ee.Initialize(project='ee-cmorton')
# points_coll = ee.FeatureCollection([
#     ee.Feature(ee.Geometry.Point(et_cell['LON'], et_cell['LAT']), {'cell_id': et_cell['GRIDMET_ID']})
#     for index, et_cell in et_cell_df.iterrows()
# ])

# huc_coll = ee.FeatureCollection('USGS/WBD/2017/HUC08')
# #     .filterBounds(points_coll.geometry().bounds())

# Map = geemap.Map()
# #Map.add_basemap("Esri.WorldImagery")
# #Map.add_basemap("OpenTopoMap")
# Map.addLayer(huc_coll, {'color': 'blue'}, 'HUC8', True, 0.25)
# Map.addLayer(points_coll, {'color': 'red'}, 'ET Cell Centroids')
# Map.centerObject(points_coll)
# Map


## Build the list of ET cells and crops to process

In [254]:
et_cell_crops = []

for cell_id, et_cell in et_cell_df.iterrows():
    for crop_num, crop_name in crop_types.items():
        kc_path = os.path.join(kc_ws, f'{et_cell["GRIDMET_ID"]}_crop_{crop_num:02d}.csv')
        if not os.path.isfile(kc_path):
            continue
        et_cell_crops.append([et_cell, kc_path])

# pprint.pprint(et_cell_crops)
for et_cell, kc_path in et_cell_crops:
    print(kc_path.split('/')[-1])


387098_crop_03.csv
387098_crop_04.csv
387098_crop_16.csv
467476_crop_03.csv
467476_crop_04.csv
467476_crop_16.csv
467476_crop_19.csv
473008_crop_03.csv
473008_crop_04.csv
473008_crop_07.csv
473008_crop_13.csv
473008_crop_16.csv
511799_crop_03.csv
511799_crop_04.csv
511799_crop_07.csv
511799_crop_16.csv
581084_crop_04.csv
605776_crop_03.csv
661167_crop_42.csv
686149_crop_19.csv
688961_crop_03.csv
564161_crop_04.csv
565566_crop_03.csv


## Functions

In [255]:
def read_daily_eto(file_path, year):
    """Read in the daily ETo"""
    daily_eto_df = pd.read_csv(file_path)
    daily_eto_df.set_index('date', inplace=True)
    daily_eto_df.index = pd.to_datetime(daily_eto_df.index, format='%Y-%m-%d')
    
    return daily_eto_df


def read_landsat_scene_etf(file_path, year):
    """Read in the Landsat scene ETf dataframe to get the Landsat observation dates"""
    scene_etf_df = pd.read_csv(file_path)
    scene_etf_df.set_index('date', inplace=True)
    scene_etf_df.index = pd.to_datetime(scene_etf_df.index, format='%Y-%m-%d')

    # Clamping would be needed if the ETf data was being used and not filled with the Kc
    # scene_etf_df['etf'] = scene_etf_df['etf'].clip(0, 1.4)
    
    return scene_etf_df
   

def interpolate_daily_etf(scene_etf_df, cell_id, year, interp_days=32):
    """"""
    
    # Build the daily dataframe with missing values that need to be interpolated
    interp_daily_etf_df = (
        scene_etf_df.drop(columns=['landsat'])
        .reindex(pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='D'), method=None)
    )
    interp_daily_etf_df['month'] = interp_daily_etf_df.index.month
    
    # Interpolate the missing daily values
    for date in pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='D'):
        # print(date)
    
        # Assume that on days with values we don't need to interpolate anything
        if not scene_etf_df[scene_etf_df.index == date].empty:
            continue
            
        prev_df = scene_etf_df[(scene_etf_df.index >= (date - timedelta(days=interp_days))) & (scene_etf_df.index < date)]
        next_df = scene_etf_df[(scene_etf_df.index > date) & (scene_etf_df.index <= (date + timedelta(days=interp_days)))]
    
        if next_df.empty and prev_df.empty:
            # No data before or after the target date
            # interp_daily_etf_df.at[date, 'etf'] = pd.nan
            pass
        elif next_df.empty and not prev_df.empty:
            # No data after the target date, so use the closest previous value
            # Is it safe to assume the dataframes are sorted by date?
            interp_daily_etf_df.at[date, 'etf'] = prev_df.iloc[-1]['etf']
        elif prev_df.empty and not next_df.empty:
            # No data before the target date, so use the closest next value
            interp_daily_etf_df.at[date, 'etf'] = next_df.iloc[0]['etf']
        else:
            # Data before and after target date, linearly interpolate value
            time_ratio = (date - prev_df.index[-1]).days / (next_df.index[0] - prev_df.index[-1]).days
            interp_value = (next_df.iloc[0]['etf'] - prev_df.iloc[-1]['etf']) * time_ratio + prev_df.iloc[-1]['etf']
            interp_daily_etf_df.at[date, 'etf'] = interp_value

    return interp_daily_etf_df


def compute_monthly_scene_counts(scene_etf_df, year):
    """Compute the scene counts per month"""
    
    scene_counts_df = scene_etf_df.groupby([scene_etf_df.index.to_period('M')]).agg(scene_count=('etf', 'count'))
    scene_counts_df.index = scene_counts_df.index.astype('datetime64[ns]')
    
    # Is there a way to fill with 0's when you reindex (instead of the manual approach here)?
    scene_counts_df = scene_counts_df.reindex(pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='MS'), method=None)
    scene_counts_df[scene_counts_df['scene_count'].isnull()] = 0
    scene_counts_df['scene_count'] = scene_counts_df['scene_count'].astype(int)
    
    # Set the month as the index for joining
    scene_counts_df['month'] = scene_counts_df.index.month
    scene_counts_df.set_index('month', inplace=True)
    
    return scene_counts_df


def compute_interpolated_monthly_et(interp_daily_etf_df):
    """"""
    
    # Join the interpolated ETf to the daily ETo
    interp_daily_df = daily_eto_df.join(interp_daily_etf_df, how='inner')
    
    # Compute daily ET
    interp_daily_df['et'] = interp_daily_df['eto'] * interp_daily_df['etf']
    
    # Compute monthly aggregations
    interp_month_df = (
        interp_daily_df.groupby([interp_daily_df.index.to_period('M')])
        .agg(et_interp=('et', 'sum'))
    )
    interp_month_df['month'] = interp_month_df.index.month
    interp_month_df.set_index('month', inplace=True)

    return interp_month_df


def compute_interpolated_daily_counts(interp_daily_etf_df):
    # TODO: We probably don't need to compute ET here and could just use ETf
    
    # Join the interpolated ETf to the daily ETo
    interp_daily_df = daily_eto_df.join(interp_daily_etf_df, how='inner')
    
    # Compute daily ET
    interp_daily_df['et'] = interp_daily_df['eto'] * interp_daily_df['etf']
    
    # Compute the daily image counts for each month
    daily_counts_df = (
        interp_daily_df.groupby([interp_daily_df.index.to_period('M')])
        .agg(daily_count=('et', 'count'))
    )
    daily_counts_df['month'] = daily_counts_df.index.month
    daily_counts_df.set_index('month', inplace=True)
    
    return daily_counts_df


def read_target_daily_kc(kc_path, year):
    """Get the target Kc curve"""
    
    # The first line of the file is the crop type
    #   or we might want to get it from the file name
    # kc_crop_num = int(kc_path.split('_')[-1])
    with open(kc_path) as f:
        kc_crop = f.readline()
    kc_crop_num = int(kc_crop.split('-')[0].replace('#', '').strip())
    kc_crop_name = kc_crop.split('-')[-1].strip()
    
    # Read in the full Kc time series
    kc_df = pd.read_csv(kc_path, skiprows=1)
    kc_df.set_index('Date', inplace=True, drop=True)
    kc_df.index = pd.to_datetime(kc_df.index, format='%Y-%m-%d')

    # Then filter to the target year    
    target_daily_kc_df = kc_df.loc[
        (kc_df.index >= datetime(year-1, 11, 1)) & (kc_df.index < datetime(year+1, 3, 1)), 
        ['Kc', 'Kcb']
    ]
    target_daily_kc_df.rename(columns={"Kc": "kc", "Kcb": "kcb"}, inplace=True)
    # target_daily_kc_df.set_index('Date', inplace=True)
    # target_daily_kc_df.index = pd.to_datetime(target_daily_kc_df.index, format='%Y-%m-%d')
    
    return target_daily_kc_df


def compute_target_month_et(target_daily_kc_df, daily_eto_df, cell_id, year):
    """Compute the target daily ET"""
    
    # Filter to the target year since the extra months are needed for interpolation
    target_daily_df = daily_eto_df.join(target_daily_kc_df, how='inner')
    target_daily_df['et'] = target_daily_df['eto'] * target_daily_df['kc']
    # print(target_daily_df)
    
    # Compute the monthly aggregations for the target data
    target_month_df = target_daily_df.groupby([target_daily_df.index.to_period('M')])\
        .agg(et_target=('et', 'sum'), eto_target=('eto', 'sum'))
    target_month_df['eto_year'] = year
    
    # Convert index back to datetimes with the day
    # target_month_df.index = target_month_df.index.astype('datetime64[ns]')
    
    # Use the month as the index
    target_month_df['month'] = target_month_df.index.month
    target_month_df.set_index('month', inplace=True)

    return target_month_df


def compute_days_in_month(year):
    return pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='MS').days_in_month


## Compute final table

In [258]:
monthly_list = []
summer_list = []
growsn_list = []
annual_list = []

kc_fill = 'kc'
# kc_fill = 'kcb'


# Iterate through the ET cell and crop combinations
for et_cell, kc_path in et_cell_crops:
    
    cell_id = et_cell['GRIDMET_ID']
    
    # TODO: Switch to a regex to get the crop number out of the Kc file path
    crop_num = int(kc_path.replace('.csv', '').split('_')[-1])
    crop_name = crop_types[crop_num]
    
    print(f'ET Cell {cell_id} - Crop {crop_num} - {crop_name}')
    print(f'  {kc_path}')

    # For now compute ETo and ETf as a pair
    # If we need additional data points we could pair different Landsat and ETo years also
    for scene_year, eto_year in [[year, year] for year in landsat_years]:
        # print(f'    Landsat Year: {scene_year},  ETo Year: {eto_year}')

        eto_path = os.path.join(
            eto_ws, f'{eto_year}', f'{eto_file_prefix}_{eto_year}_{cell_id}.csv'
        )
        etf_path = os.path.join(
            etf_ws, f'{scene_year}', f'{etf_file_prefix}_{scene_year}_{cell_id}.csv'
        )

        scene_etf_df = read_landsat_scene_etf(etf_path, year=scene_year)
        #print(scene_etf_df)
    
        target_daily_kc_df = read_target_daily_kc(kc_path, year=eto_year)
        #print(target_daily_kc_df)
        #fig, ax0 = plt.subplots(1, 1, figsize=(6, 3), dpi=150)
        #target_daily_kc_df['kc'].plot(ax=ax0, color='gray', lw=0.5)
        #target_daily_kc_df['kcb'].plot(ax=ax0, color='blue', lw=0.5)

        # Fill scene ETf values with the Kc curve values
        # TODO: Make this into a separate function once the Kc tables are built
        scene_etf_df = scene_etf_df.join(target_daily_kc_df, how='inner')
        scene_etf_df['etf'] = scene_etf_df[kc_fill]
        scene_etf_df.drop(['kc'], axis=1, inplace=True)
        scene_etf_df.drop(['kcb'], axis=1, inplace=True)
        
        interp_daily_etf_df = interpolate_daily_etf(
            scene_etf_df, cell_id=cell_id, year=scene_year, interp_days=interp_days
        )
        #fig, ax0 = plt.subplots(1, 1, figsize=(6, 3), dpi=150)
        #interp_daily_etf_df['etf'].plot(ax=ax0, linestyle='none', marker='.', ms=1, color='blue')
        #scene_etf_df['etf'].plot(ax=ax0, linestyle='none', marker='^', ms=3, color='red')
        
        scene_counts_df = compute_monthly_scene_counts(scene_etf_df, year=scene_year)
        #print(scene_counts_df)
        
        daily_eto_df = read_daily_eto(eto_path, year=eto_year)
        #print(daily_eto_df)
        
        interp_month_df = compute_interpolated_monthly_et(interp_daily_etf_df)
        #print(interp_month_df)
        
        daily_counts_df = compute_interpolated_daily_counts(interp_daily_etf_df)
        #print(daily_counts_df)
        
        target_month_df = compute_target_month_et(
            target_daily_kc_df, daily_eto_df, cell_id=cell_id, year=eto_year
        )
        #print(target_month_df)
        
        # Link/join the separate dataframes 
        cell_month_df = interp_month_df.join(target_month_df, how='inner')
        cell_month_df = cell_month_df.join(daily_counts_df, how='inner')
        cell_month_df = cell_month_df.join(scene_counts_df, how='inner')
        cell_month_df['scene_year'] = eto_year
        
        # Month isn't needed as an index anymore
        cell_month_df['month'] = cell_month_df.index
        cell_month_df.reset_index(inplace=True, drop=True)
        # print(cell_month_df)

        
        # Compute temporal aggregations before missing months are dropped to get the "total" values
        # TODO: Is there a better way to get a dataframe back from the .sum() call?
        cell_summer_df = (
            cell_month_df.loc[cell_month_df['month'].isin([6, 7, 8]), ['et_target', 'eto_target']]
            .sum().to_frame().transpose()
            .rename(columns={"eto_target": "eto_total", "et_target": "et_total"})
        )
        cell_growsn_df = (
            cell_month_df.loc[cell_month_df['month'].isin(growsn_months), ['et_target', 'eto_target']]
            .sum().to_frame().transpose()
            .rename(columns={"eto_target": "eto_total", "et_target": "et_total"})
        )
        cell_annual_df = (
            cell_month_df[[ 'et_target', 'eto_target']]
            .sum().to_frame().transpose()
            .rename(columns={"eto_target": "eto_total", "et_target": "et_total"})
        )
        
        
        # Remove months that do not have enough daily data points (missing 4+ days) 
        # Applying this after temporal aggregations above
        # Later on test removing months that are missing any daily data points
        # TODO: Check what the OpenET interpolation is doing for this
        cell_month_df['days_in_month'] = compute_days_in_month(year=eto_year)
        cell_month_df = cell_month_df[cell_month_df['daily_count'] >= 25]
        # cell_month_df = cell_month_df[cell_month_df['daily_count'] >= (cell_month_df['days_in_month'] - 3)]

        
        # Recompute temporal aggregations after missing months are dropped to get the "target" sums
        columns = ['et_interp', 'et_target', 'eto_target', 'month_count']
        cell_month_df['month_count'] = 1
        cell_summer_masked_df = cell_month_df.loc[cell_month_df['month'].isin([6, 7, 8]), columns].sum().to_frame().transpose()
        cell_growsn_masked_df = cell_month_df.loc[cell_month_df['month'].isin(growsn_months), columns].sum().to_frame().transpose()
        cell_annual_masked_df = cell_month_df[columns].sum().to_frame().transpose()
        cell_summer_masked_df['month_count'] = cell_summer_masked_df['month_count'].astype(int)
        cell_growsn_masked_df['month_count'] = cell_growsn_masked_df['month_count'].astype(int)
        cell_annual_masked_df['month_count'] = cell_annual_masked_df['month_count'].astype(int)
        cell_month_df.drop(['month_count'], axis=1, inplace=True)

        # Add the masked sums to the aggregation dataframes
        cell_summer_df = pd.concat([cell_summer_df, cell_summer_masked_df], axis=1)
        cell_growsn_df = pd.concat([cell_growsn_df, cell_growsn_masked_df], axis=1)
        cell_annual_df = pd.concat([cell_annual_df, cell_annual_masked_df], axis=1)


        # Add all the other cell properties
        # TODO: Find a better way to add the cell data
        cell_month_df['cell_id'] = cell_id
        cell_month_df['crop_num'] = crop_num
        cell_month_df['crop_name'] = crop_name
        cell_month_df['cell_huc'] = et_cell['HUC8']
        cell_month_df['cell_lat'] = et_cell['LAT']
        cell_month_df['cell_lon'] = et_cell['LON']

        # TODO: Come up with a cleaner way of building these
        cell_summer_df['scene_year'] = eto_year
        cell_summer_df['cell_id'] = cell_id
        cell_summer_df['crop_num'] = crop_num
        cell_summer_df['crop_name'] = crop_name
        cell_summer_df['cell_huc'] = et_cell['HUC8']
        cell_summer_df['cell_lat'] = et_cell['LAT']
        cell_summer_df['cell_lon'] = et_cell['LON']

        cell_growsn_df['scene_year'] = eto_year
        cell_growsn_df['cell_id'] = cell_id
        cell_growsn_df['crop_num'] = crop_num
        cell_growsn_df['crop_name'] = crop_name
        cell_growsn_df['cell_huc'] = et_cell['HUC8']
        cell_growsn_df['cell_lat'] = et_cell['LAT']
        cell_growsn_df['cell_lon'] = et_cell['LON']

        cell_annual_df['scene_year'] = eto_year
        cell_annual_df['cell_id'] = cell_id
        cell_annual_df['crop_num'] = crop_num
        cell_annual_df['crop_name'] = crop_name
        cell_annual_df['cell_huc'] = et_cell['HUC8']
        cell_annual_df['cell_lat'] = et_cell['LAT']
        cell_annual_df['cell_lon'] = et_cell['LON']

        # print('\nMonth data')
        # print(cell_month_df)
        # print('\nSummer Months Sums')
        # print(cell_summer_df)
        # print('\nGrowing Season Months Sums')
        # print(cell_growsn_df)
        # print('\nAll Months Sums')
        # print(cell_annual_df)

        # Save the values for each cell
        monthly_list.append(cell_month_df)
        summer_list.append(cell_summer_df)
        growsn_list.append(cell_growsn_df)
        annual_list.append(cell_annual_df)

        #break
        
    #break

monthly_df = pd.concat(monthly_list)
summer_df = pd.concat(summer_list)
growsn_df = pd.concat(growsn_list)
annual_df = pd.concat(annual_list)

# print(monthly_df.head())

print('\nDone')

ET Cell 387098 - Crop 3 - Alfalfa Hay
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/387098_crop_03.csv
ET Cell 387098 - Crop 4 - Grass Hay
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/387098_crop_04.csv
ET Cell 387098 - Crop 16 - Grass Pasture
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/387098_crop_16.csv
ET Cell 467476 - Crop 3 - Alfalfa Hay
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/467476_crop_03.csv
ET Cell 467476 - Crop 4 - Grass Hay
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/467476_crop_04.csv
ET Cell 467476 - Crop 16 - Grass Pasture
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/467476_crop_16.csv
ET Cell 467476 - Crop 19 - Apples and Cherries
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/467476_crop_19.csv
ET Cell 473008 - Crop 3 - Alfalfa Hay
  /Users/Charles.Morton@dri.edu/Projects/interpolation_assessment/kc/

# Compute Statistics

In [276]:
stats_str_fmt = '{:6s} {:>12s}  {:>12s}'
stats_int_fmt = '{:6s} {:>12d}  {:>12d}'
stats_flt_fmt = '{:6s} {:>12.6f}  {:>12.6f}'

def print_stats(data_a, data_b, band_1='et_target', band_2='et_interp'):
    # Print comparison statistics of the for two dataframes
    diff_a = data_a[band_1] - data_a[band_2]
    diff_b = data_b[band_1] - data_b[band_2]
    
    # Check if this is the correct calcualtion for computing slope through the origin
    slope_a = (data_a[band_1] * data_a[band_2]).sum() / (data_a[band_2] * data_a[band_2]).sum()
    slope_b = (data_b[band_1] * data_b[band_2]).sum() / (data_b[band_2] * data_b[band_2]).sum()
    r2_a = data_a[[band_1, band_2]].corr()[band_2].values[0]
    r2_b = data_b[[band_1, band_2]].corr()[band_2].values[0]
    rmse_a = (diff_a ** 2).mean() ** 0.5 
    rmse_b = (diff_b ** 2).mean() ** 0.5

    print(stats_int_fmt.format('n:', diff_a.count(), diff_b.count()))
    print(stats_flt_fmt.format('slope:', slope_a, slope_b))
    print(stats_flt_fmt.format('r2:', r2_a, r2_b))
    print(stats_flt_fmt.format('rmse:', rmse_a, rmse_b))
    print(stats_flt_fmt.format('mae:', diff_a.abs().mean(), diff_b.abs().mean()))
    print(stats_flt_fmt.format('mbe:', diff_a.mean(), diff_b.mean()))

monthly_two_landsat_df = monthly_df[monthly_df['scene_year'].isin(two_landsat_years)]
monthly_one_landsat_df = monthly_df[monthly_df['scene_year'].isin(one_landsat_years)]

summer_two_landsat_df = summer_df[summer_df['scene_year'].isin(two_landsat_years)]
summer_one_landsat_df = summer_df[summer_df['scene_year'].isin(one_landsat_years)]
growsn_two_landsat_df = growsn_df[growsn_df['scene_year'].isin(two_landsat_years)]
growsn_one_landsat_df = growsn_df[growsn_df['scene_year'].isin(one_landsat_years)]
annual_two_landsat_df = annual_df[annual_df['scene_year'].isin(two_landsat_years)]
annual_one_landsat_df = annual_df[annual_df['scene_year'].isin(one_landsat_years)]


## Statistics for all monthly data points

In [277]:
print('All Data')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(monthly_two_landsat_df, monthly_one_landsat_df)
# two_landsat_df.plot.scatter('et_target', 'et_interp', s=1, alpha=0.5)

All Data
          2 Landsat     1 Landsat
n:             3903          3457
slope:     1.002557      1.002226
r2:        0.991395      0.977622
rmse:      8.671639     13.760669
mae:       5.742038      8.907141
mbe:       1.282735      1.938637


## Statistics by scene count in month

In [278]:
print('Scene count >= 0\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] >= 0],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] >= 0]
)

Scene count >= 0

          2 Landsat     1 Landsat
n:             3903          3457
slope:     1.002557      1.002226
r2:        0.991395      0.977622
rmse:      8.671639     13.760669
mae:       5.742038      8.907141
mbe:       1.282735      1.938637


In [279]:
print('Scene count >= 1\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] >= 1],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] >= 1]
)

Scene count >= 1

          2 Landsat     1 Landsat
n:             3669          3058
slope:     1.002223      1.000724
r2:        0.991380      0.980396
rmse:      8.680792     12.901879
mae:       5.768813      8.455374
mbe:       1.154035      1.664622


In [280]:
print('Scene count >= 2\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] >= 2],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] >= 2]
)

Scene count >= 2

          2 Landsat     1 Landsat
n:             3131          2052
slope:     1.003492      1.004299
r2:        0.991927      0.984267
rmse:      8.432069     11.451939
mae:       5.677367      7.713916
mbe:       1.183576      1.716468


In [281]:
print('Scene count >= 3\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] >= 3],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] >= 3]
)

Scene count >= 3

          2 Landsat     1 Landsat
n:             2496           918
slope:     1.003069      1.003996
r2:        0.992116      0.987396
rmse:      8.341381      9.984178
mae:       5.620720      6.904197
mbe:       1.127301      1.375896


In [282]:
print('\nStats - scene count >= 4\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] >= 4],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] >= 4]
)


Stats - scene count >= 4

          2 Landsat     1 Landsat
n:             1733           205
slope:     1.001892      0.996694
r2:        0.992880      0.986561
rmse:      7.872635      8.757895
mae:       5.377493      6.191969
mbe:       1.052586      0.145805


## Check each scene count separately

In [283]:
print('Scene count == 0\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] == 0],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] == 0]
)

Scene count == 0

          2 Landsat     1 Landsat
n:              234           399
slope:     1.061126      1.028888
r2:        0.952746      0.939133
rmse:      8.526835     19.100955
mae:       5.322210     12.369562
mbe:       3.300697      4.038736


In [284]:
print('Scene count == 1\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] == 1],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] == 1]
)

Scene count == 1

          2 Landsat     1 Landsat
n:              538          1006
slope:     0.976752      0.986905
r2:        0.976058      0.966002
rmse:     10.006343     15.442981
mae:       6.301006      9.967771
mbe:       0.982115      1.558868


In [285]:
print('Scene count == 2\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] == 2],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] == 2]
)

Scene count == 2

          2 Landsat     1 Landsat
n:              635          1134
slope:     1.006815      1.004608
r2:        0.988446      0.981389
rmse:      8.779464     12.514672
mae:       5.900030      8.369402
mbe:       1.404775      1.992169


In [286]:
print('Scene count == 3\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] == 3],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] == 3]
)

Scene count == 3

          2 Landsat     1 Landsat
n:              763           713
slope:     1.007202      1.006616
r2:        0.989327      0.987329
rmse:      9.318857     10.309794
mae:       6.173161      7.108975
mbe:       1.297000      1.729569


In [287]:
print('Scene count >= 4\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['scene_count'] >= 4],
    monthly_one_landsat_df[monthly_one_landsat_df['scene_count'] >= 4]
)

Scene count >= 4

          2 Landsat     1 Landsat
n:             1733           205
slope:     1.001892      0.996694
r2:        0.992880      0.986561
rmse:      7.872635      8.757895
mae:       5.377493      6.191969
mbe:       1.052586      0.145805


## Compute the stats for the summer months (individually)

In [288]:
print('Summer months (individually)\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['month'].isin([6, 7, 8])],
    monthly_one_landsat_df[monthly_one_landsat_df['month'].isin([6, 7, 8])]
)

Summer months (individually)

          2 Landsat     1 Landsat
n:             1035          1016
slope:     0.999779      0.999384
r2:        0.970897      0.910079
rmse:     11.429280     19.090292
mae:       8.247443     13.282178
mbe:       0.767159      1.893579


## Compute stats for the growing season months (individually)

In [289]:
print('Growing season months (individually)\n')
print(stats_str_fmt.format('', '2 Landsat', '1 Landsat'))
print_stats(
    monthly_two_landsat_df[monthly_two_landsat_df['month'].isin([4, 5, 6, 7, 8, 9, 10])],
    monthly_one_landsat_df[monthly_one_landsat_df['month'].isin([4, 5, 6, 7, 8, 9, 10])]
)

Growing season months (individually)

          2 Landsat     1 Landsat
n:             2414          2365
slope:     1.003036      1.003940
r2:        0.983838      0.961019
rmse:     10.092026     15.653645
mae:       7.074853     10.647948
mbe:       1.266080      2.433173


## Compute the stats for the summer, growing season, and annual totals 

In [290]:
print('Summer total ET')
print('       {:>10s}  {:>10s}'.format('2 Landsat', '1 Landsat'))
print_stats(summer_two_landsat_df, summer_one_landsat_df)

Summer total ET
        2 Landsat   1 Landsat
n:              345           345
slope:     1.002884      1.007545
r2:        0.984892      0.963012
rmse:     21.120094     33.482643
mae:      16.155975     25.231994
mbe:       2.301478      5.576453


In [291]:
print('Growing season total ET')
print('       {:>10s}  {:>10s}'.format('2 Landsat', '1 Landsat'))
print_stats(growsn_two_landsat_df, growsn_one_landsat_df)

Growing season total ET
        2 Landsat   1 Landsat
n:              345           345
slope:     1.009761      1.017526
r2:        0.986440      0.972080
rmse:     30.104611     47.090567
mae:      23.936674     37.399012
mbe:       8.858892     16.679575


In [292]:
print('Annual total ET')
print('       {:>10s}  {:>10s}'.format('2 Landsat', '1 Landsat'))
print_stats(annual_two_landsat_df, annual_one_landsat_df)

Annual total ET
        2 Landsat   1 Landsat
n:              345           345
slope:     1.015324      1.019579
r2:        0.984008      0.969601
rmse:     35.861764     52.120380
mae:      28.271379     41.918877
mbe:      14.511641     19.425707


## Compute the stats for the same time periods, but compare to the total ET computed without missing months

In [235]:
# print('Summer total ET')
# print('       {:>10s}  {:>10s}'.format('2 Landsat', '1 Landsat'))
# print_stats(summer_two_landsat_df, summer_one_landsat_df, band_1='et_total', band_2='et_interp')

In [236]:
# print('Growing season total ET')
# print('       {:>10s}  {:>10s}'.format('2 Landsat', '1 Landsat'))
# print_stats(growsn_two_landsat_df, growsn_one_landsat_df, band_1='et_total', band_2='et_interp')

In [237]:
# print('Annual total ET')
# print('       {:>10s}  {:>10s}'.format('2 Landsat', '1 Landsat'))
# print_stats(annual_two_landsat_df, annual_one_landsat_df, band_1='et_total', band_2='et_interp')